## Polymorphism and Code Reuse

I guess "Polymorphism and Code Reuse" is kind of redundant. According to Ye Olde Wikipedia, *polymorphism is the provision of a single interface to entities of different types or the use of a single symbol to represent multiple different types,* So it's code reuse related to types, which is exactly what I'm going to talk about. I guess macros are also a kind of code reuse, but this isn't about that.

Julia allows a fully dynamic approach to typing if desired, though it is strongly recommended to use types when defining structs for performance reason. We have already seen many function/method definitions which are dynamically typed.

This means functions are polymorphic by default. Like Python, you can send any type to any function, and you won't get an error until you hit an interface which isn't provided for that type. Julia could theoretically catch these kinds of errors earlier, but it doesn't do that, since compilation of functions happens at runtime the first time a function is required, not prior to any code execution.

Generally speaking, idiomatic Julia specifies functions as dynamically as possible and uses type declarations for leveraging multiple dispatch. However, if you want to get your type errors earlier than later (never a bad thing), you may also add declarations for that, though they will still only be caught at runtime (just potentially much earlier).

However, using type declarations does not mean that one forgoes polymorphism in Julia. Julia supports ad-hoc polymorphism with multiple dispatch, parametric polymorphism with generics and first-class type constructors, and subtyping with abstract types. Of course, all these features can be used together to create well-typed, but highly polymorphic functions and structs.

### Inheritance with Abstract Types

If you're familiar with classical object orientated languages, or even a prototypical language like JS, you may be glad (or not) to learn that Julia also has inheritance. You may be displeased (or not) to learn that it's upside-down from how it works in classical OO--but at least it isn't prototypical.

Let's return to our rectangles, but we'll add a cirle, too.

In [1]:
"""Types which inherit from `Shape` should provide an
`area` method.
"""
abstract type Shape end
combined_area(a::Shape, b::Shape) = area(a) + area(b)

struct Circle <: Shape
    diameter::Float64
end
radius(c::Circle) = c.diameter / 2
area(c::Circle) = π * radius(c) ^ 2

"""Types which inherit from `AbstractRectangle should
provide `height` and `width` methods.
"""
abstract type AbstractRectangle <: Shape end
area(r::AbstractRectangle) = width(r) * height(r) 

struct Rectangle <: AbstractRectangle
    width::Float64
    height::Float64
end
width(r::Rectangle) = r.width
height(r::Rectangle) = r.height
 
struct Square <: AbstractRectangle
    length::Float64
end
width(s::Square) = s.length
height(s::Square) = s.length

height (generic function with 2 methods)

Here, we've declared two abstract types. `Shape`, which is at the top of this heirarchy (well, it's a subtype of `Any`, like everything else), and AbstractRectangle, which is a subtype of `Shape`.

You should note that abstract types in Julia have no data layout. They also have no instances. They exist purely for the inheritance of function dispatches. Subtypes of `Shape` can use its `combined_area` method, provided they have their own `area` methods. Subtypes of `AbstractRectangle` can use its `area` method, provided they have their own `height` and `width` methods.

`Circle` inherits directly from `Shape` and implements an `area` method derived from its diameter.

`Rectangle` and `Square` are both subtypes of `AbstractRectangle` and inherit its `area` method, implementing their own `width` and `height` methods. One thing that annoys me in Julia is that there's no way to enforce the implementation of certain interfaces on subtypes, so document it well! One possible benefit to this is that you don't actually need to implement interfaces for methods of a supertype you don't intend to use. I'd rather have an enforcible contract, but them's the breaks. In typical Julia style, you get a runtime error about missing methods instead. *\*sigh\**

Anyway:

In [2]:
c = Circle(3)
s = Square(3)
r = Rectangle(3, 2)

@show combined_area(c, s)
@show combined_area(s, r);

combined_area(c, s) = 16.068583470577035
combined_area(s, r) = 15.0


Though the abstract types are used in polymorphic way, Julia will still compile efficient code for the various types required.

### Composition and method forwarding: an alternative to inheritance.

One common complaint about the lack of classes in Julia is that structs cannot inherit data fields from super classes. I'm of the opinion that over-zealous use inheritance (particularly of data layout) tends to result in spaghetti code, but I suppose there are cases where it's useful. If you want to use inheritance in Julia, you may have to keep writing the same field-name over and over in all the structs. I think this is good because it's more explicit. Your mileage may vary.

In any case, composition is generally considered a superior pattern to inheritance, at least in part because it's more explicit. The strategy for compositional inheritance is to create structs that have the type with the required interface as a field rather than a super type, and then optionally forward the appropriate methods to that field. Julia makes this easier with `eval`, since you can forward methods with a loop. It is typical in Julia to use `eval` for preprocessing of this sort.

In [3]:
struct HasInterestingField{T}
    data::T
end
 
double(hif::HasInterestingField) = hif.data * 2
shout(hif::HasInterestingField) = uppercase(string(hif.data, "!"))
 
# the compositional way add those attibutes to another struct.
struct WantsInterestingField{T}
    interesting::HasInterestingField{T}
    WantsInterestingField(data) = new(HasInterestingField(data))
end
 
# forward methods

for method in (:double, :shout)
    @eval $method(wif::WantsInterestingField) = $method(wif.interesting)
end

# same as:
#     double(wif::WantsInterestingField) = double(wif.interesting)
#     shout(wif::WantsInterestingField) = shout(wif.interesting)

ErrorException: syntax: too few type parameters specified in "new{...}"

So, `eval` forwards the methods and the inner constructor insures that users `WantsInterestingField` don't have to care that one of its members is `HasInterestingField`. This could (and perhaps should) be an outer constructor, if it was believed that there were a case where it would be useful to build a `WantsInterestingField` instace with a pre-existing instance of `HasInterestingField`, which could certainly happen. In addition to using `eval`, one could also write a macro that works for the forwarding of (kinda) arbitrary methods. This is one I wrote, that uses the attribute as the first argument and you may feel free to steal it:

In [4]:
# forward methods on a struct to a field of that struct.
# good for your composition.
# syntax: @forward CompositeType.attr Base.iterate Base.length :*
# Symbol literals automatically become Base.:symbol. Good for adding
# methods to built-in operators.
macro forward(property, functions...)
    structname = property.args[1]
    field = property.args[2].value
    block = quote end
    for f in functions
        # case for operators
        if f isa QuoteNode
            f = :(Base.$(f.value)) 
            def1 = :($f(x::$structname, y) = $f(x.$field, y))
            def2 = :($f(x, y::$structname) = $f(x, y.$field))
            push!(block.args, def1, def2)
        # case for other stuff
        else
            def = :($f(x::$structname, args...; kwargs...) = $f(x.$field, args...; kwargs...))
            push!(block.args, def)
        end
    end
    esc(block)
end

# demo:
struct Foo
    x::String
end

@forward Foo.x Base.string :* :^

foo = Foo("foo")

@show string(foo, "!")
@show foo * "!"
@show foo ^ 4;

string(foo, "!") = "foo!"
foo * "!" = "foo!"
foo ^ 4 = "foofoofoofoo"


It's part of a library I use privately under a 2-clause BSD liscense. You may copy it freely but it comes with no warranty. There is also an undocumented `@forward` macro in the popular (and cool) [Lazy.jl](https://github.com/MikeInnes/Lazy.jl) package which many people use. It is probable that it's more robust than what's posted above. Mike Innes, who wrote it, is a genius.

### Generics: Statically typed dynamic typing

Generics aren't really a form of dynamic typing. However, languages like OCaml and Haskell that use them ubiquitously almost feel like they are dynamically typed, but with the safety guarantees of statically typed languages. In Julia, generics can be used for safety, but they are also the key to making structs that are both efficient and polymorphic.

Comming back to our `Point` struct, let's say we wanted to be able to use any type to represent `x` and `y`, but we still wanted the efficiency of inferable types. That's where generics come in:

In [5]:
struct GenPoint{T}
    x::T
    y::T
end

GenPoint(1, 3)

GenPoint{Int64}(1, 3)

What we've done is create a _type constructor_ which takes a type as a parameter in curly braces, from which instances can then be created. Julia allows the type parameter to be ommited when using the constructor because it can be inferred from the arguments. In this particular case, x and y must be of the same type because the type constructor only has one generic parameter.

In [6]:
GenPoint(1, 3.0)

MethodError: MethodError: no method matching GenPoint(::Int64, ::Float64)
Closest candidates are:
  GenPoint(::T, !Matched::T) where T at In[5]:2

As you can see, both arguments must be of the same type. However, type constructors can have additional generic parameters:

In [7]:
struct GenPoint2{X,Y}
    x::X
    y::Y
end

GenPoint2(1, 3.0)

GenPoint2{Int64,Float64}(1, 3.0)

Having x and y of a point be the same actually seems like a good idea, but there's still a problem with our first generic point implementation--It's maybe *too* generic.

In [8]:
GenPoint("foo", "bar")

GenPoint{String}("foo", "bar")

Obviously a point containing strings is stupid, and it's going to break all kinds of assumptions about the operations we can apply to x and y. What else can we do?

In [9]:
struct RealPoint
    x::Real
    y::Real
end

RealPoint(0x5, 0xaa)

RealPoint(0x05, 0xaa)

Here, we used the abstract type `Real` as the type of our fields. However, you shouldn't actually do this. Because `Real` isn't a concrete type, the Julia compiler can't make any assumptions about the data layout of `RealPoint`, and significant method selection will have to occur at runtime rather than compiletime. The proper way to do this would be as follows:

In [10]:
struct Point{T<:Real}
    x::T
    y::T
end

@show Point(1, 3)
@show Point(1.4, 2.5);

Point(1, 3) = Point{Int64}(1, 3)
Point(1.4, 2.5) = Point{Float64}(1.4, 2.5)


What we've done here is say that generic `T` must be a subtype of `Real`, so we can prevent senseless values like this:

In [11]:
Point("foo", "bar")

MethodError: MethodError: no method matching Point(::String, ::String)

Generics are especially useful for implementing containers. Let's try a linked list. First code, then explanations:

In [12]:
const Opt = Union{T,Nothing} where T

struct List{T}
    head::T
    tail::Opt{List{T}}
end

head(l::List) = l.head
tail(l::List) = l.tail

# built a list from an array
buildlist(array::AbstractArray{T}, n) where T =
    if n == lastindex(array)
        List{T}(array[n], nothing)
    else
        List(array[n], buildlist(array, n+1), array[n])
    end

buildlist(array) = buildlist(array, firstindex(array))

# implement the iteration protocol
Base.iterate(l::List) = iterate(l, l)
Base.iterate(::List, l::List) = head(l), tail(l)
Base.iterate(::List, ::Nothing) = nothing

# demo:
list = buildlist([1,2,3,4,5])
@show list

for val in list
    println(val)
end

list = List{Int64}(1, List{Int64}(2, List{Int64}(3, List{Int64}(4, List{Int64}(5, nothing)))))
1
2
3
4
5


Ok, that's a lot of code Let's start at line 1. The `where` keyword is a way to specify that an expression contains a generic (it's not necessary when declaring structs because it's obvious what it means). `Opt` is simply a type constructor alias now.

In [13]:
Opt

Union{Nothing, T} where T

This is used frequently in the standard library. For example, `Vector` is just a partially parameterized `Array`:

In [14]:
Vector

Array{T,1} where T

A type constructor alias like this can be made into a concrete type by supplying the needed type parameters:

In [15]:
Opt{Int64}

Union{Nothing, Int64}

Making a type constructor alias was probably not strictly necessary here because it was only used once, but this snippet finds its way into a lot of my code because it's helpful for implementing all kinds of recursive data structures.

Next, lines 3-6. A node in the list must have a value of type `T` and a tail which can be another node of the same type or `nothing`.

Lines 12-19 are simple, fairly standard way to build up a linked list from an array using recursion. The only interesting part from the perspective of generics is the function declaration. The only interesting thing is line 12, where we declare the generic T in the function signature:

```julia
buildlist(array::AbstractArray{T}, n) where T
```

This turns whatever the element type of the array is into a value in the function body. This value is used to create the end of the list, to ensure that the List has the same type as the array. For example:

In [16]:
buildlist([1, "foo"])

List{Any}(1, List{Any}("foo", nothing))

This would not work otherwise, because the initail list would default to the type of its value, `String` in this case, and it would be unable to instantiate an `Int64` node on top of it. We can also use this strategy to implement a `typeof` function.

In [17]:
whichtype(::T) where T = T
whichtype("foo")

String

However, the fact that it's so easy to do means that you rarely actually need to use `typeof` outside of the REPL.

Lines 22-24 implement the [iteration protocol](https://docs.julialang.org/en/v1/base/collections/#lib-collections-iteration-1) by adding methods to Base.iterate for our type. Read that link to understand the iteration protocol, and then notice line 23. `Base.iterate(::List, l::List)`. The first argument in this case isn't even given a name because its value will never be used. It's just there to make sure the correct dispatch is selected. This will come back again later with the trait pattern. Line 24 uses the same pattern implement the terminal case. If the `state` argument of `iterate` is `nothing`, it means we have exhausted all the nodes, we return `nothing` as well (instead of an iterable-state 2-tuple) to signal that iteration is finished.

And that's basically it. That's how generics work in Julia.

### The Holy Trait Pattern

Traits are sort of a contract with the compiler that a certain type is going to implement the right interfaces, and can therefore be used as an argument for any function that requires that interface. It is an alternative to classical inheritance which is superior, in my opinion. A class normally only has one super type, but a type could implement many traits. Yes, there is multiple inheritance, but that's the highway to hell, whereas traits tend to keep the spaghetti monster at bay. Traits in Rust, Interfaces in Go and type classes in Haskell are all examples of Traits in different languages.

Julia doesn't have traits as a language-level feature, but there is a popular pattern for implementing traits which occurs frequently in the standard library, called the Holy Trait. While this is an awesome name, it is not so called because it exhibits exceptional holiness--if anything, it's more given to sin. Rather, it gets its name from the gentleman who suggested it in a github issue--Julia contributer, Tim Holy.

You can read a bit about this pattern [here](https://docs.julialang.org/en/v1/manual/methods/#Trait-based-dispatch-1) in the official docs, and the [github discussion](https://github.com/JuliaLang/julia/issues/2345#issuecomment-54537633) where it came up originally. One area you might need to use one of the traits in the standard library is when implementing your own iterable type. There is class of traits called `Base.IteratorSize` that can be used to give hints about the size of your iterable so that space can be allocated more efficiently when broadcasting and mapping and things. For example, as it stands now, there is no method for determining the length of the above linked list implementation, so we could add the `Base.SizeUnknown` trait.

```julia
Base.IteratorSize(::List) = Base.SizeUnkown()
```

However, we could also implement a length function:

In [18]:
Base.length(l::List) = 1 + length(tail(l))
Base.length(::Nothing) = 0
# probably overriding Base.length for type Nothing is a bad idea.
# I a better choice would be to create your own Nil type for the List.

Base.IteratorSize(::List) = Base.HasLength()

This will then be used in various functions around the standard library for efficient allocation. Of course, there is still a question about how efficient it can possibly be to have to traverse the list just to get its length, but that's one of several reasons why one should avoid linked lists in performance-sensitive code. It's just an example, man.

Anyway, the standard library documentation for traits will effectively show you how to use the traits it provides, but it oddly stops short of actually showing you how to create your own traits. In its simplest form, a trait is just an empty struct and a dispatch that throws an error in the most general case. You then check for the trait by using the constructor in a method dispatch.

In [19]:
struct Zlurmable end
Zlurmable(::T) where T = error("Type $T doesn't implement the Zlurmable trait")

zlurm(x) = zlurm(Zlurmable(x), x)
zlurm(::Zlurmable, x) = x + 1

zlurm (generic function with 2 methods)

However, there are no types (yet) that have our `Zlurmable` trait, so it's impossible to use the `zlurm` function.

In [20]:
zlurm(1)

ErrorException: Type Int64 doesn't implement the Zlurmable trait

What we need to do is add the trait to a type:

In [21]:
Zlurmable(::Int64) = Zlurmable()
zlurm(3)

4

Traits in Julia are essentially an (uninforced) promise that your type is going to implement the correct interfaces. Because Julia code is dynamically typed by default, you could simply take a devil-may-care approach to it and just write your function generically and not care about safeties like traits. However, there are obvious benefits when you add inheritance into the mix. This allows different ways to implement the trait's interface and lets methods optimize against different types of traits. If we were to implement our own version of `Base.IteratorSize`, it might look like this (actually, it looks exactly like this):

In [22]:
abstract type IteratorSize end
struct SizeUnknown <: IteratorSize end
struct HasLength <: IteratorSize end
struct HasShape{N} <: IteratorSize end
struct IsInfinite <: IteratorSize end

IteratorSize(::Any) = HasLength()

IteratorSize

Here, there are a variety of ways to think about an iterator's size, and different dispatches can make the most of that information. For some reason, the default `IteratorSize` is set to `HasLength`. Not sure why they chose to do this, because what happens is it assumes your iterable implements a length function and then crashes when it doesn't find one, rather than letting you know you need to implement `IteratorSize(::MyType)` with one of the four options, since there are plenty of cases where the size of an iterator is unknown. (like, whenever dealing with IO)

So why are types used to implement traits rather than other kinds of values? That's an easy one: Types are known at compile-time. Values are only known at runtime. This means that traits are computed only when the dispatch compiles and have no cost thereafter.

## Closing Thoughts

Composition and encapsulation are great. Polymorphism is fantastic. I hope this guide gives you some idea about how to use the features of Julia design fast, flexible and maintainable software if you come from a background of classical inheritance and/or dynamic typing. If you have any suggestions or corrections, please feel free to open a github issue. Also, let me know if there's something you didn't understand! I try to write in a way that is simple to understand, but these are somewhat abstract ideas, and it's easy to slip into jargon once you get comfortable with them.